In [1]:
!pip install lyricsgenius

  Using cached https://files.pythonhosted.org/packages/9e/d4/10f46e5cfac773e22707237bfcd51bbffeaf0a576b0a847ec7ab15bd7ace/beautifulsoup4-4.6.0-py3-none-any.whl
ERROR: textract 1.6.3 has requirement beautifulsoup4==4.8.0, but you'll have beautifulsoup4 4.6.0 which is incompatible.
  Found existing installation: beautifulsoup4 4.8.0
    Uninstalling beautifulsoup4-4.8.0:
      Successfully uninstalled beautifulsoup4-4.8.0


In [109]:
#import lyricsgenius as lg
import string
import time
import requests
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup

In [115]:
class SSP():
    
    def clean(s):
        s = s.replace('&','and')
        punc = string.punctuation.replace('\"','')
        punc = punc.replace('\'','')
        for x in punc:
            clean1 = s.split(x)
            clean = clean1[0]
            if clean is None:
                return None
            if re.match("^[a-zA-Z0-9_ ]*$", clean):
                clean = clean.strip()
                return clean
        
    
    def get_url(songartist_url):
        url = 'http://www.absolutelyrics.com/lyrics/view/{}'.format(songartist_url)
        return url
        
    def get_lyrics(url):
        #time.sleep(2)
        page = requests.get(url)#headers={'User-Agent': 'Div class lyrics (adamgoldsteindatascience@gmail.com)'}
        #Scrape the song lyrics from the HTML
        
        html = BeautifulSoup(page.text, "html.parser")
        div = html.find(id="view_lyrics")
        #if not div:
        #    return None # Sometimes the lyrics section isn't found

        # Scrape lyrics if proper section was found on page
        if div is None:
            return None
        
        lyrics = div.get_text()
        # Remove [Verse], [Bridge], etc.
        #lyrics = re.sub('(\[.*?\])*', '', lyrics)
        #lyrics = re.sub('\n', ' ', lyrics)  # Gaps between verses
        return lyrics

        
        
        
        
        

In [116]:
file = "Data/8_sample_entire.csv"
Playlist_track_lyrics = []
i = 0
delimiter = ':'
lyrics_kickout=[]

start = time.time()
#genius = lyricsgenius.Genius("CLMJDhT2HYCbiXT9grKoh3LZ2hjiCRuvYYxTOj3lARJgM03frK2k739jf9UwDxFs")

with open(file,'r') as csvfile:
    musicfile = csv.reader(csvfile,delimiter='|')
    next(musicfile) #Skip header row
    head = [next(musicfile) for x in range(4500)]
    
    for line in head:
        i+=1
        track_uri = line[3]
        track_name = SSP.clean(line[4])
        artist_name = SSP.clean(line[1])


        if track_name == None or artist_name == None:
            kickout = {'track_uri':track_uri,'issue': track_name}
            lyrics_kickout.append(kickout)
            print(i,track_uri,kickout)
            continue

        song_artist = '/'.join([artist_name,track_name])
        song_artist_url = song_artist.replace(' ','_')
        link = SSP.get_url(song_artist_url)
        
        lyrics = SSP.get_lyrics(link)
        print(i,True,link)
        Playlist_track_lyrics.append({'artist_name':artist_name,'track_uri':track_uri,'track_name':track_name
                                  ,'lyrics':lyrics})



            



end = time.time()
duration = ((end - start)/60)

1 True http://www.absolutelyrics.com/lyrics/view/Mike_Reed/Grace_Under_Pressure
2 1qQudQbHnq56UGuTndqsnQ {'track_uri': '1qQudQbHnq56UGuTndqsnQ', 'issue': 'Sullivan'}
3 True http://www.absolutelyrics.com/lyrics/view/William_Aura/Timeless_I
4 True http://www.absolutelyrics.com/lyrics/view/Journey/Natural_Thing
5 True http://www.absolutelyrics.com/lyrics/view/Gare_Du_Nord/Gimme_That_Sexy_Thing
6 True http://www.absolutelyrics.com/lyrics/view/Nanci_Griffith/Roses_on_the_4th_of_July
7 True http://www.absolutelyrics.com/lyrics/view/Andrea_Bocelli/Amapola
8 True http://www.absolutelyrics.com/lyrics/view/The_Love_Unlimited_Orchestra/Fix_You
9 True http://www.absolutelyrics.com/lyrics/view/Robert_Plant/Carving_up_the_World_Again
10 True http://www.absolutelyrics.com/lyrics/view/Elvis_Presley/Memories
11 True http://www.absolutelyrics.com/lyrics/view/Pratt_and_McClain/Dancing_In_the_City
12 True http://www.absolutelyrics.com/lyrics/view/Scott_Johnson/Iron_Wave_Cap_Thugs
13 True http://www.absolu

105 True http://www.absolutelyrics.com/lyrics/view/Hans_Zimmer/Honor_Him
106 5vICxX1XkpkaZgsSADhoHc {'track_uri': '5vICxX1XkpkaZgsSADhoHc', 'issue': None}
107 True http://www.absolutelyrics.com/lyrics/view/The_Beatles/And_Here_We_Are_Again
108 4GLlMtmMGKdDFFcnk8JnwN {'track_uri': '4GLlMtmMGKdDFFcnk8JnwN', 'issue': 'Play A Simple Melody'}
109 True http://www.absolutelyrics.com/lyrics/view/Itzhak_Perlman/Kreisler
110 True http://www.absolutelyrics.com/lyrics/view/Fearless_Motivation/Reinvent_Yourself
111 True http://www.absolutelyrics.com/lyrics/view/Twenty_One_Pilots/Ode_to_Sleep
112 4dizR5ps7OK8uVY2bUcXLb {'track_uri': '4dizR5ps7OK8uVY2bUcXLb', 'issue': None}
113 True http://www.absolutelyrics.com/lyrics/view/George_Frideric_Handel/Joshua
114 True http://www.absolutelyrics.com/lyrics/view/Wall_Of_Voodoo/Good_Times
115 0kLOAyvEvNlqio9q1Y92LY {'track_uri': '0kLOAyvEvNlqio9q1Y92LY', 'issue': None}
116 True http://www.absolutelyrics.com/lyrics/view/Jimmy_Smith/Autumn_Leaves
117 6Hji9NVapyY

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [102]:
duration

3.8500783165295918

In [108]:
df = pd.DataFrame(Playlist_track_lyrics)
df.dropna().count()
#df.to_csv('Playlist_track_lyrics.csv',sep=':',index=False)

artist_name    46
track_uri      46
track_name     46
lyrics         46
dtype: int64

In [99]:
duration

3.8500783165295918

In [100]:
a = 'adfadfadfaf'
a = a.split(':')
a[0]

'adfadfadfaf'

In [78]:
type(string.punctuation)

str

In [ ]:
b = pd